# Python on-the-fly colormap generator

Useful bit of code that allows you to see how python plots look on-the-fly.

In [ ]:
import Make_EQ as eq
from ipywidgets import interactive, VBox, Layout, widgets, fixed
import matplotlib.pyplot as plt
import numpy as np
from scipy import interpolate
import os
import xarray as xr

In [ ]:
# define output interpolation specifics
varfile = '/home/jovyan/work/products/dataset-oc-glo-chl-multi_cci-l4-chl_4km_monthly-rep-v02_1568876553803.nc'
varname = 'CHL'
lonname = 'longitude'
latname = 'latitude'

varfile = '/home/jovyan/work/products/METOFFICE-GLO-SST-L4-NRT-OBS-SST-V2_1568885414480.nc'
varname = 'analysed_sst'
lonname = 'lon'
latname = 'lat'

bbox = [5, 20, -35, -15]

In [ ]:
# open data
ds1 = xr.open_dataset(varfile)
LAT = ds1.variables[latname].values
LON = ds1.variables[lonname].values
VAR = ds1.variables[varname].values
ds1.close()

if len(np.shape(VAR)) > 2:
    VAR = np.squeeze(VAR[0,:,:])
    
if varname == 'analysed_sst':
    VAR = VAR - 273.15

ii = np.where((LON >= bbox[0]) & (LON <= bbox[1]))[0]
jj = np.where((LAT >= bbox[2]) & (LAT <= bbox[3]))[0]

LON = LON[ii[0]:ii[-1]]
LAT = LAT[jj[0]:jj[-1]]
VAR = VAR[jj[0]:jj[-1],ii[0]:ii[-1]]

vlimits = [int(np.nanmin(VAR))-1, int(np.nanmax(VAR))+1, 255]

In [ ]:
# colourmap examples

# Medspiration
channel_red=[1,1,0,0,0,0,0,0.5,1,1,1,0.25]
channel_green=[1,0.4,0,0.5,1,0.7,0.42,0.7,1,0.5,0,0]
channel_blue=[1,1,0.2,0.6,1,0.5,0,0,0,0,0,0]
    
#channel_red = [1,0.5,0,1]
#channel_green = [1,1,0.5,1]
#channel_blue = [1,0.5,0,1]

In [ ]:
%matplotlib inline

nchan = len(channel_red)
print('nChannels: '+str(nchan))

# build the widgets
red_widgets, green_widgets, blue_widgets = eq.make_widgets(channel_red, channel_green, channel_blue)

# build the plot command
runCMD = eq.build_plot_command(vlimits,\
                               red_widgets, green_widgets, blue_widgets)
# run the plot command
exec(runCMD)
box_layout = Layout(display='flex', flex_flow='row', justify_content='space-between', align_items='center')

# display the plot
for ii in range(0, nchan):
    display(VBox([iplot.children[ii], iplot.children[ii+nchan],iplot.children[ii+nchan*2]],\
                  layout=box_layout))
display(iplot.children[-1])

In [ ]:
# interpolate colour map onto required range / steps
reds = interactive_plot.result[2]
greens = interactive_plot.result[3]
blues = interactive_plot.result[4]
tscale_orig = np.linspace(vmin, vmax, len(reds))
tscale_new = np.linspace(vmin, vmax, vsteps)

interp_redf = interpolate.interp1d(tscale_orig, reds)
interp_greenf = interpolate.interp1d(tscale_orig, greens)
interp_bluef = interpolate.interp1d(tscale_orig, blues)

interp_red = np.array(interp_redf(tscale_new)*255.).astype(int)
interp_green = np.array(interp_greenf(tscale_new)*255.).astype(int)
interp_blue = np.array(interp_bluef(tscale_new)*255.).astype(int)

In [ ]:
# write to file
my_file = 'Test_colbar.grass'
if os.path.exists(my_file):
    os.remove(my_file)
    
with open(my_file, 'w') as the_file:
    for ii in range(0,len(interp_red)):
        the_file.write(str(tscale_new[ii]) + ' ' + \
                       str(interp_red[ii]) + ' ' + \
                       str(interp_green[ii]) + ' ' + \
                       str(interp_blue[ii]))
        if ii != len(interp_red)-1:
            the_file.write('\n')